In [7]:
from src.libraryAnalysier import LibraryAnalysier

import pandas as pd
import os
import numpy as np

from tqdm import tqdm

In [8]:
data_dir = './data/100x20x3/'
OUTPUT_PATH = 'D:\\Musik\\Genres\\'
DIR_PATH = 'D:\\Musik\\Genres\\DnB\\'
DIR_ERROR_PATH = DIR_PATH

In [9]:
from keras.models import load_model
reconstructed_model = load_model(os.path.join(data_dir, "model.h5"))

import joblib
loaded_scaler = joblib.load(os.path.join(data_dir, "scaler.pkl"))  # load from disk

import pickle
with open(os.path.join(data_dir, "labels.pkl"), "rb") as a_file:
    GENRES = pickle.load(a_file)

In [10]:
print(GENRES)

{0: 'Dancefloor', 1: 'Deep', 2: 'Jungle', 3: 'Liquid', 4: 'Minimal', 5: 'Neurofunk', 6: 'Sambass', 7: 'Shriek', 8: 'Techno'}


In [11]:
files = os.listdir(DIR_PATH)
liblys = LibraryAnalysier(library="D:\\Musik\\Archive")

        
for file in files:
    __bar = tqdm(total=liblys.segments, desc=f"{os.path.basename(file)}")
    if(os.path.splitext(file)[-1] not in ['.wav', '.mp3']):
        print('not supported')
        continue
    file_path = os.path.join(DIR_PATH, file)

    result = liblys.loadFile(file_path, __bar) 
    # sis = Analysis(audio_path=file_path, options=config)

    inside_df = pd.DataFrame(result)
    inside_df.drop(['length'], axis=1, inplace=True)
    inside_df = pd.DataFrame(loaded_scaler.transform(inside_df), columns=inside_df.columns)
    inside_df.reset_index(drop=True)
    q = reconstructed_model.predict(inside_df).T

    
    points = [np.mean(r, axis=None) for r in q]
    biggest_number = max(points)

    guess = 'other'
    guess_save = 0
    alt_guess = 'other'
    
    temp = 0
    for i, r in enumerate(q):
        _max = np.max(r)
        temp += _max
        g = np.mean(r) * (1 - (np.var(r)))
        
        if((_max < 0.80)):
            continue
        if(g > guess_save):
            alt_guess = guess
            alt_guess_save = guess_save
            guess_save = g
            guess = GENRES[i]
        elif(g > alt_guess_save):
            alt_guess = GENRES[i]
            alt_guess_save = g

    # print('Avage: ', temp - guess_save)

    path_dir = os.path.join(OUTPUT_PATH, guess)
    path_file = os.path.join(DIR_PATH, file)
    new_file = os.path.join(path_dir, os.path.basename(file))
    
    # if(guess_save < 0.8):
    #     continue
    
    # bar__.close()
    # print()
    if alt_guess != 'other':    
        print(f' > ({guess}: {round(np.max(guess_save)*100, 2)}% or {alt_guess}: {round(np.max(alt_guess_save)*100, 2)}%)')
    else:
        print(f' > ({guess}: {round(np.max(guess_save)*100, 2)}%)')

    # if not os.path.exists(OUTPUT_PATH):
    #     os.makedirs(OUTPUT_PATH)
    # if not os.path.exists(path_dir):
    #     os.makedirs(path_dir)
    
    # print(path_file, new_file)
    # os.replace(path_file, new_file)

SyntaxError: invalid syntax (1756794151.py, line 28)